In [2]:
!pip install neuralcoref

    100% |████████████████████████████████| 296kB 8.6MB/s ta 0:00:01
    100% |████████████████████████████████| 10.2MB 4.8MB/s eta 0:00:01
    100% |████████████████████████████████| 3.7MB 9.5MB/s eta 0:00:01
    100% |████████████████████████████████| 184kB 2.3MB/s ta 0:00:011
    100% |████████████████████████████████| 2.1MB 8.9MB/s eta 0:00:01
    100% |████████████████████████████████| 122kB 38.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 31.9MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
!pip install spacy==2.1.0
!python3 -m spacy download en

    100% |████████████████████████████████| 27.7MB 1.5MB/s ta 0:00:011
    100% |████████████████████████████████| 92kB 34.1MB/s ta 0:00:01
    100% |████████████████████████████████| 3.2MB 6.9MB/s ta 0:00:011
    100% |████████████████████████████████| 2.1MB 6.6MB/s ta 0:00:011
en-core-web-sm 2.2.0 has requirement spacy>=2.2.0, but you'll have spacy 2.1.0 which is incompatible.
  Found existing installation: preshed 3.0.2
    Uninstalling preshed-3.0.2:
      Successfully uninstalled preshed-3.0.2
  Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Found existing installation: thinc 7.1.1
    Uninstalling thinc-7.1.1:
      Successfully uninstalled thinc-7.1.1
  Found existing installation: spacy 2.2.1
    Uninstalling spacy-2.2.1:
      Successfully uninstalled spacy-2.2.1
You are using pip version 10.0.1, however version 19.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100

In [55]:
# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')
import re
from nltk.tokenize import sent_tokenize

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp,greedyness=0.5,max_dist=50,blacklist=False)

# Alternatively you can use the black sheep story as text instead of "Scientists" text below
# with open('black_sheep.txt', "r") as file:
#     text = file.read()

text = "Scientists know many things about the Sun. They know how old it is. \
The Sun is more than 4½ billion years old. It is also a star that is the centre of our solar system. \
They also know the Sun’s size."

text = str(text)

doc = nlp(text)

clusters = doc._.coref_clusters

print("clusters ",clusters)
print ("\n\n")
resolved_coref = doc._.coref_resolved
print ("Resolved by NeuralCoref: \n" )
print(resolved_coref)


# As you can see from the printed output of resolved_coref, the pronoun resolved neural co-reference text is very noisy. 
# So we took the get_resolved function and modified it to suit our use case. We only replace the below list of pronouns.

pronoun_list=['he','she','it','they','them','him','her','his','hers','its','we','us']

# Given an index eg: 8 and a list ['Scientists ', 'know ', 'many ', 'things ', 'about ', 'the ', 'Sun', '. ', 'They ', 'know ', 'how ', 'old ', 'it ', 'is', '. ', 'The ', 'Sun ', 'is ', 'more ', 'than ', '4½ ', 'billion ', 'years ', 'old', '. ', 'It ', 'is ', 'also ', 'a ', 'star', '. ', 'They ', 'also ', 'know ', 'the ', 'Sun', '’s ', 'size', '.']
# Retrieve the sentence corresponding to that index as a list. Eg: ['They ', 'know ', 'how ', 'old ', 'it ', 'is', '. '] 
def get_sentence_for_index(index,resolved_list):
    end_of_sentence_punctuation = ['.','!','?']
    beginning_index = index
    ending_index = index
    while (beginning_index>=0):
        val = resolved_list[beginning_index].strip()
        if (val in end_of_sentence_punctuation):
            break
        else:
            beginning_index = beginning_index -1
    while (ending_index<=(len(resolved_list)-1)):
        val = resolved_list[ending_index].strip()
        if (val in end_of_sentence_punctuation):
            break
        else:
            ending_index = ending_index +1
    return resolved_list[beginning_index+1:ending_index+1],beginning_index+1


def get_resolved(doc, clusters):
    ''' Return a list of utterrances text where the coref are resolved to the most representative mention'''
    resolved = list(tok.text_with_ws for tok in doc)
    questions =[]
    for cluster in clusters:
        for coref in cluster:
            coref_text = coref.text.lower()
            cluster_main_text = cluster.main.text.lower()
            if len(coref_text.split())==1 and coref_text != cluster_main_text and coref_text in pronoun_list:
                get_sentence,start_index = get_sentence_for_index(coref.start,resolved)
                get_sentence_string = ''.join(get_sentence).lower()
                cluster_main_string = ' '.join(cluster_main_text.strip().split())
                if (cluster_main_string not in get_sentence_string):
                    resolved[coref.start] = cluster.main.text + doc[coref.end-1].whitespace_
                    if start_index == coref.start:
                        resolved[coref.start] = resolved[coref.start].capitalize() 
                    final_sentence = ''.join(get_sentence)
                    # Leave out very short sentences to frame questions.
                    if len(final_sentence)>20:
                        questions.append([final_sentence,coref.text,resolved[coref.start]])

                

    return ''.join(resolved),questions

resolved,questions = get_resolved (doc, clusters)
print("\n\n")
print ("Custom resolved: \n")
print (resolved)
print("\n")



clusters  [Scientists: [Scientists, They, They], the Sun: [the Sun, The Sun, Sun, It]]



Resolved by NeuralCoref: 

Scientists know many things about the Sun. Scientists know how old it is. the Sun the Sun is more than 4½ billion years old. the Sun is also a star that is the centre of our solar system. Scientists also know the Sun’s size.



Custom resolved: 

Scientists know many things about the Sun. Scientists know how old it is. The Sun is more than 4½ billion years old. The sun is also a star that is the centre of our solar system. Scientists also know the Sun’s size.




In [57]:
print ("\nQuestions generated :")
print ("[Note: There might be a few answer errors because of the errors in the coreference algorithm itself] \n")

for index,question in enumerate(questions):
    print ('%d) What does \"%s\" refer to in the sentence - \"%s\"?'%(index+1,question[1],question[0].strip()))
    print ("Ans : %s\n"%(question[2]))






Questions generated :
[Note: There might be a few answer errors because of the errors in the coreference algorithm itself] 

1) What does "They" refer to in the sentence - "They know how old it is."?
Ans : Scientists 

2) What does "They" refer to in the sentence - "They also know the Sun’s size."?
Ans : Scientists 

3) What does "It" refer to in the sentence - "It is also a star that is the centre of our solar system."?
Ans : The sun 

